# IMPORTS

## Libraries

In [1]:
import pandas as pd
import numpy as np
import random
import datetime
import pickle

import warnings
warnings.filterwarnings("ignore")

import seaborn as sns
import matplotlib.pyplot as plt
from IPython.core.display import HTML

from sklearn.preprocessing import RobustScaler, MinMaxScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso

import xgboost as xgb
from lightgbm import LGBMRegressor

## Helper Functions

In [2]:
def crossValidation(XTraining, kfold, modelName, model='default', verbose=False):
    maeList = []
    mapeList = []
    rmseList = []

    for k in reversed(range(1, kfold+1)):
        if verbose:
            print(f'\nKFold Number: {k}')
        # Start and End Date for Validation
        startDateValid = XTraining['Date'].max() - datetime.timedelta(days=k*6*7)
        endDateValid = XTraining['Date'].max() - datetime.timedelta(days=(k-1)*6*7)

        # Filtering Dataset
        training = XTraining[XTraining['Date'] < startDateValid]
        validation = XTraining[(XTraining['Date'] >= startDateValid) & (XTraining['Date'] <= endDateValid)]

        # Training and Validation Dataset
        # Training
        XKFoldTraining = training.drop(['Date', 'Sales'], axis=1)
        yKFoldTraining = training['Sales']

        # Validation
        XKFoldValidation = validation.drop(['Date', 'Sales'], axis=1)
        yKFoldValidation = validation['Sales']

        # Model
        ## Model Map
        modelMap = {
            'Linear Regression': LinearRegression(),
            'Lasso': Lasso(alpha=0.01),
            'Random Forest Regressor': RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42),
            'XGBoost Regressor': xgb.XGBRegressor( objective='reg:squarederror', n_estimators=500, eta=0.01, max_depth=10, 
                                                      subsample=0.7, colsample_bytree=0.9),
            'Lightgbm Regressor': LGBMRegressor(num_leaves=10, min_data_in_leaf=50, n_jobs=-1, random_state=42, n_estimators=500)   
        }
        
        ## Mapped Model
        if model == 'default':
            model = modelMap[modelName]
        else: model = model
        
        model.fit(XKFoldTraining, yKFoldTraining)

        # Prediction
        yhat = model.predict(XKFoldValidation)

        #Performance
        modelResult = mlError('Linear Regression', np.expm1(yKFoldValidation), np.expm1(yhat))
        
        #Store Performance of each KFold iteration
        maeList.append(modelResult['MAE'].tolist())
        mapeList.append(modelResult['MAPE'].tolist())
        rmseList.append(modelResult['RMSE'].tolist())


    dictResult = {
                    'Model Name': [modelName],
                    'MAE CV': [np.round(np.mean(maeList),2).astype(str) + ' +/- ' + np.round(np.std(maeList),2).astype(str)],
                    'MAPE CV': [np.round(np.mean(mapeList),2).astype(str) + ' +/- ' + np.round(np.std(mapeList),2).astype(str)],
                    'RMSE CV': [np.round(np.mean(rmseList),2).astype(str) + ' +/- ' + np.round(np.std(rmseList),2).astype(str)]
                }

    return pd.DataFrame(dictResult)


def mean_percentage_error( y, yhat ):
    return np.mean( ( y - yhat ) / y )

def mean_absolute_percentage_error(y, yhat):
    return np.mean(np.abs((y - yhat) / y))


def mlError(modelName, y, yhat):
    mae = mean_absolute_error(y, yhat)
    mape = mean_absolute_percentage_error(y, yhat)
    rmse = np.sqrt(mean_squared_error(y, yhat))
    
    return pd.DataFrame({
                            'ModelName': modelName,
                            'MAE': mae,
                            'MAPE': mape,
                            'RMSE': rmse,
                        }, index=[0])



def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 16]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()

In [3]:
jupyter_settings()

Populating the interactive namespace from numpy and matplotlib


## Loading Data

In [6]:
dfDataPreparation = pd.read_csv('../../01-Data/Results/01-FirstRoundCRISP/dfFeatureEngineering.csv', low_memory=False, parse_dates=['Date'])

# Data Preparation

In [7]:
dfRaw1 = dfDataPreparation.copy()

## Rescaling

In [8]:
numAttributes = dfRaw1.select_dtypes(include=['int64', 'float64'])

rs = RobustScaler()
mms = MinMaxScaler()

#Competion Distance >> Presence of well defined outiliers
numAttributes['CompetitionDistance'] = rs.fit_transform(numAttributes[['CompetitionDistance']].values)
pickle.dump(rs, open('D:/01-DataScience/04-Projetos/00-Git/Rossmann-Store-Sales/02-Notebooks/01-FirstRoundCRISP/parameter/CompetitionDistanceScaler.pkl', 'wb'))


#Competion Time Month >> Presence of well defined outiliers
numAttributes['CompetionTimeMonth'] = rs.fit_transform(numAttributes[['CompetionTimeMonth']].values)
pickle.dump(rs, open('D:/01-DataScience/04-Projetos/00-Git/Rossmann-Store-Sales/02-Notebooks/01-FirstRoundCRISP/parameter/CompetionTimeMonthScaler.pkl', 'wb'))


#Promo Time Week
numAttributes['PromoTimeWeek'] = mms.fit_transform(numAttributes[['PromoTimeWeek']].values)
pickle.dump(mms, open('D:/01-DataScience/04-Projetos/00-Git/Rossmann-Store-Sales/02-Notebooks/01-FirstRoundCRISP/parameter/PromoTimeWeekScaler.pkl', 'wb'))


#Year
numAttributes['Year'] = mms.fit_transform(numAttributes[['Year']].values)
pickle.dump(mms, open('D:/01-DataScience/04-Projetos/00-Git/Rossmann-Store-Sales/02-Notebooks/01-FirstRoundCRISP/parameter/YearScaler.pkl', 'wb'))

FileNotFoundError: [Errno 2] No such file or directory: 'D:/01-DataScience/04-Projetos/00-Git/Rossmann-Store-Sales/02-Notebooks/01-FirstRoundCRISP/parameter/CompetitionDistanceScaler.pkl'

## Transformation

In [ ]:
#State Holiday -> One Hot Encoding
dfRaw1 = pd.get_dummies(dfRaw1, prefix=['StateHoliday'], columns=['StateHoliday'])

#Store Type -> Label Encoding
le = LabelEncoder()
dfRaw1['StoreType'] = le.fit_transform(dfRaw1['StoreType'])
pickle.dump(le, open('D:/01-DataScience/04-Projetos/00-Git/Rossmann-Store-Sales/02-Notebooks/01-FirstRoundCRISP/parameter/StoreTypeScaler.pkl', 'wb'))

#Assortment -> Ordinal Encoding
dictAssortment = {
                    'basic': 1,
                    'extra': 2,
                    'extended': 3
                    }
dfRaw1['Assortment'] = dfRaw1['Assortment'].map(dictAssortment)

# Rossmann Class